<a href="https://colab.research.google.com/github/cpptake/CommonLit/blob/main/roberta_large_maunish_BASELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisite

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Jul 19 12:56:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    56W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install same version of library as Kaggle Notebook

In [3]:
# # cp でrequirementsをカレントdirにコピー
# !cp -f /content/drive/MyDrive/kaggle/commonlit/roberta-large/requirements.txt ./
# !cat ./requirements.txt

In [4]:
!pip uninstall -r /content/drive/MyDrive/CommonLit/working/requirements.txt -y

Uninstalling pandas-1.2.3:
  Successfully uninstalled pandas-1.2.3
Uninstalling sklearn-0.0:
  Successfully uninstalled sklearn-0.0
Uninstalling sklearn-pandas-2.1.0:
  Successfully uninstalled sklearn-pandas-2.1.0
Uninstalling torch-1.7.0:
  Successfully uninstalled torch-1.7.0
Uninstalling torchmetrics-0.2.0:
  Successfully uninstalled torchmetrics-0.2.0
Uninstalling torchvision-0.8.1:
  Successfully uninstalled torchvision-0.8.1
Uninstalling transformers-4.5.1:
  Successfully uninstalled transformers-4.5.1


In [5]:
!pip install -r /content/drive/MyDrive/CommonLit/working/requirements.txt

  Using cached https://files.pythonhosted.org/packages/f3/d4/3fe3b5bf9886912b64ef040040aec356fa48825e5a829a84c2667afdf952/pandas-1.2.3-cp37-cp37m-manylinux1_x86_64.whl
Processing /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074/sklearn-0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/35/52/bb537702967ad30ed17bcda71c9e163777a696c1a982da421dd2489b4ff7/sklearn_pandas-2.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d9/74/d52c014fbfb50aefc084d2bf5ffaa0a8456f69c586782b59f93ef45e2da9/torch-1.7.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3a/42/d984612cabf005a265aa99c8d4ab2958e37b753aafb12f31c81df38751c8/torchmetrics-0.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a3/39/a9caac0deb027feec2cdd7cc40b2a598256d3f50050c80f349c030f915f2/torchvision-0.8.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.

In [6]:
!pip freeze |grep -e random -e math -e numpy -e pandas -e torch -e transformers -e sklearn -e gc

mpmath==1.2.1
numpy==1.19.5
pandas==1.2.3
pandas-datareader==0.9.0
pandas-gbq==0.13.3
pandas-profiling==1.4.1
sklearn==0.0
sklearn-pandas==2.1.0
tensorflow-gcs-config==2.5.0
torch==1.7.0
torchmetrics==0.2.0
torchsummary==1.5.1
torchtext==0.10.0
torchvision==0.8.1
transformers==4.5.1


## Prepare dataset

### kaggle.json

In [7]:
# !mkdir -p /root/.kaggle/
# !cp ./drive/MyDrive/kaggle/commonlit/kaggle.json ~/.kaggle/kaggle.json
# !chmod 600 ~/.kaggle/kaggle.json

!pip install -q kaggle
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/Colab\ Notebooks/kaggle.json /root/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


### Competition dataset

In [8]:
# !mkdir -p /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/
# !kaggle competitions download -c commonlitreadabilityprize -p /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/
# !cp -f ./drive/MyDrive/kaggle/commonlit/train_stratiKfold.csv.zip /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/

In [9]:
# !unzip -o ../input/commonlitreadabilityprize/train.csv.zip -d ../input/commonlitreadabilityprize/
# !unzip -o ../input/commonlitreadabilityprize/train_stratiKfold.csv.zip -d ../input/commonlitreadabilityprize/

In [10]:
!ls ../input/commonlitreadabilityprize/

### Pretrained RoBERTa Large 
- Pretrain RoBERTa Large in the same way as this notebook
  - https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain
- Dataset:
  - https://www.kaggle.com/iamnishipy/roberta-large-20210712191259-mlm

In [11]:
!mkdir -p /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/pretrained-model/
!kaggle datasets download iamnishipy/roberta-large-20210712191259-mlm

401 - Unauthorized


In [12]:
!unzip -o ./roberta-large-20210712191259-mlm.zip -d ../input/commonlitreadabilityprize/pretrained-model/

unzip:  cannot find or open ./roberta-large-20210712191259-mlm.zip, ./roberta-large-20210712191259-mlm.zip.zip or ./roberta-large-20210712191259-mlm.zip.ZIP.


In [13]:
!ls ../input/commonlitreadabilityprize/pretrained-model/

ls: cannot access '../input/commonlitreadabilityprize/pretrained-model/': No such file or directory


# Overview
This is kernel is almost the same as [Lightweight Roberta solution in PyTorch](https://www.kaggle.com/andretugan/lightweight-roberta-solution-in-pytorch), but instead of "roberta-base", it starts from [Maunish's pre-trained model](https://www.kaggle.com/maunish/clrp-roberta-base).

Acknowledgments: some ideas were taken from kernels by [Torch](https://www.kaggle.com/rhtsingh) and [Maunish](https://www.kaggle.com/maunish).

In [14]:
!pip install transformers accelerate datasets

In [15]:
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AdamW
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.model_selection import KFold

import gc
gc.enable()

## Prepare dataset

In [16]:
NUM_FOLDS = 5
NUM_EPOCHS = 3
BATCH_SIZE = 8
MAX_LEN = 248
EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
ROBERTA_PATH = "/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-large"
TOKENIZER_PATH = "/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-large"
# ROBERTA_PATH = "/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large"
# TOKENIZER_PATH = "/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large"
# ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"
# TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

In [18]:
train_df = pd.read_csv("/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/train.csv")

# Remove incomplete entries if any.
train_df.drop(train_df[(train_df.target == 0) & (train_df.standard_error == 0)].index,
              inplace=True)
train_df.reset_index(drop=True, inplace=True)

test_df = pd.read_csv("/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/sample_submission.csv")

In [19]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

# Dataset

In [20]:
class LitDataset(Dataset):
    def __init__(self, df, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only
        self.text = df.excerpt.tolist()
        #self.text = [text.replace("\n", " ") for text in self.text]
        
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)        
    
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, target)

# Model
The model is inspired by the one from [Maunish](https://www.kaggle.com/maunish/clrp-roberta-svm).

In [21]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)
        #https://towardsdatascience.com/attention-based-deep-multiple-instance-learning-1bb3df857e24
        # 768: node fully connected layer
        # 512: node attention layer
        # self.attention = nn.Sequential(            
        #     nn.Linear(768, 512),            
        #     nn.Tanh(),                       
        #     nn.Linear(512, 1),
        #     nn.Softmax(dim=1)
        # )        

        # self.regressor = nn.Sequential(                        
        #     nn.Linear(768, 1)                        
        # )


        #768 -> 1024
        #512 -> 768
        self.attention = nn.Sequential(            
            nn.Linear(1024, 768),            
            nn.Tanh(),                       
            nn.Linear(768, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        # roberta_output = self.roberta(input_ids=input_ids,
        #                               attention_mask=attention_mask)
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)
        
        last_hidden_state = roberta_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask

        # print(mean_embeddings.shape)

        logits = self.regressor(mean_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)

        return preds

        #### nishipy original ####
        # # There are a total of 13 layers of hidden states.
        # # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # # We take the hidden states from the last Roberta layer.
        # last_layer_hidden_states = roberta_output.hidden_states[-1]

        # # The number of cells is MAX_LEN.
        # # The size of the hidden state of each cell is 768 (for roberta-base).
        # # In order to condense hidden states of all cells to a context vector,
        # # we compute a weighted average of the hidden states of all cells.
        # # We compute the weight of each cell, using the attention neural network.
        # weights = self.attention(last_layer_hidden_states)
                
        # # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # # Now we compute context_vector as the weighted average.
        # # context_vector.shape is BATCH_SIZE x 768
        # context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # # Now we reduce the context vector to the prediction score.
        # return self.regressor(context_vector)

In [22]:
def eval_mse(model, data_loader):
    """Evaluates the mean squared error of the |model| on |data_loader|"""
    model.eval()            
    mse_sum = 0

    with torch.no_grad():
        for batch_num, (input_ids, attention_mask, target) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)                        
            target = target.to(DEVICE)           
            
            pred = model(input_ids, attention_mask)                       

            mse_sum += nn.MSELoss(reduction="sum")(pred.flatten(), target).item()
                

    return mse_sum / len(data_loader.dataset)

In [23]:
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

In [24]:
def train(model, model_path, train_loader, val_loader,
          optimizer, scheduler=None, num_epochs=NUM_EPOCHS):    
    best_val_rmse = None
    best_epoch = 0
    step = 0
    last_eval_step = 0
    eval_period = EVAL_SCHEDULE[0][1]    

    start = time.time()

    for epoch in range(num_epochs):                           
        val_rmse = None         

        for batch_num, (input_ids, attention_mask, target) in enumerate(train_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)            
            target = target.to(DEVICE)                        

            optimizer.zero_grad()
            
            model.train()

            pred = model(input_ids, attention_mask)
                                                        
            mse = nn.MSELoss(reduction="mean")(pred.flatten(), target)
                        
            mse.backward()

            optimizer.step()
            if scheduler:
                scheduler.step()
            
            if step >= last_eval_step + eval_period:
                # Evaluate the model on val_loader.
                elapsed_seconds = time.time() - start
                num_steps = step - last_eval_step
                print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                last_eval_step = step
                
                val_rmse = math.sqrt(eval_mse(model, val_loader))                            

                print(f"Epoch: {epoch} batch_num: {batch_num}", 
                      f"val_rmse: {val_rmse:0.4}")

                for rmse, period in EVAL_SCHEDULE:
                    if val_rmse >= rmse:
                        eval_period = period
                        break                               
                
                if not best_val_rmse or val_rmse < best_val_rmse:                    
                    best_val_rmse = val_rmse
                    best_epoch = epoch
                    torch.save(model.state_dict(), model_path)
                    print(f"New best_val_rmse: {best_val_rmse:0.4}")
                else:       
                    print(f"Still best_val_rmse: {best_val_rmse:0.4}",
                          f"(from epoch {best_epoch})")                                    
                    
                start = time.time()
                                            
            step += 1
                        
    
    return best_val_rmse

In [25]:
#怪しい
def create_optimizer(model):
    #model.named_parameters():
    #Base -> 205
    #Large -> 397
    named_parameters = list(model.named_parameters())    

    #Base
    # roberta_parameters = named_parameters[:197]    
    # attention_parameters = named_parameters[199:203]
    # regressor_parameters = named_parameters[203:]
    
    #Large
    roberta_parameters = named_parameters[:389]    
    attention_parameters = named_parameters[391:395]
    regressor_parameters = named_parameters[395:]
        
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]

    parameters = []
    parameters.append({"params": attention_group})
    parameters.append({"params": regressor_group})

    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if "bias" in name else 0.01
        lr = 2e-5
        #roberta-base: 
        # if layer_num >= 69: #4/12layers       
        #     lr = 5e-5
        # if layer_num >= 1f33: #8/12layers
        #     lr = 1e-4
        #roberta-large
        if layer_num >= 133: #8/24layers     
            lr = 5e-5
        if layer_num >= 261: #16/24layers
            lr = 1e-4


        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})

    return AdamW(parameters)

In [26]:
# SEED = 1000
# kfold = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
# for fold, (train_indices, val_indices) in enumerate(kfold.split(train_df)):
#     print(fold)
#     print('------------')
#     print(val_indices)

# Debug

In [26]:
model = LitModel().to(DEVICE)

In [27]:
named_parameters = list(model.named_parameters())
len(named_parameters)

397

In [ ]:
# for name, param in model.named_parameters():
#     print(name)

# Train


In [27]:
gc.collect()

SEED = 1000
list_val_rmse = []

kfold = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kfold.split(train_df)):    
    print(f"\nFold {fold + 1}/{NUM_FOLDS}")
    model_path = f"model_{fold + 1}.pth"
        
    set_random_seed(SEED + fold)
    
    train_dataset = LitDataset(train_df.loc[train_indices])    
    val_dataset = LitDataset(train_df.loc[val_indices])    
        
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              drop_last=True, shuffle=True, num_workers=2)    
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=2)    
        
    set_random_seed(SEED + fold)    
    
    model = LitModel().to(DEVICE)
    
    optimizer = create_optimizer(model)                        
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_training_steps=NUM_EPOCHS * len(train_loader),
        num_warmup_steps=50)    
    
    list_val_rmse.append(train(model, model_path, train_loader,
                               val_loader, optimizer, scheduler=scheduler))

    del model
    gc.collect()
    
    print("\nPerformance estimates:")
    print(list_val_rmse)
    print("Mean:", np.array(list_val_rmse).mean())
    


Fold 1/5

16 steps took 6.91 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9267
New best_val_rmse: 0.9267

16 steps took 6.38 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7674
New best_val_rmse: 0.7674

16 steps took 6.44 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7594
New best_val_rmse: 0.7594

16 steps took 6.43 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6386
New best_val_rmse: 0.6386

16 steps took 6.39 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.7786
Still best_val_rmse: 0.6386 (from epoch 0)

16 steps took 6.41 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6306
New best_val_rmse: 0.6306

16 steps took 6.41 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5816
New best_val_rmse: 0.5816

16 steps took 6.42 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6318
Still best_val_rmse: 0.5816 (from epoch 0)

16 steps took 6.38 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6653
Still best_val_rmse: 0.5816 (from epoch 0)

16 steps took 6.45 seconds
Epoch: 0 batch_num: 160 val_rmse: 0.8283
Still best_val_rmse: 

# Inference

In [28]:
test_dataset = LitDataset(test_df, inference_only=True)

In [31]:
all_predictions = np.zeros((len(list_val_rmse), len(test_df)))

test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for index in range(len(list_val_rmse)):            
    model_path = f"model_{index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModel()
    model.load_state_dict(torch.load(model_path))    
    model.to(DEVICE)
    
    all_predictions[index] = predict(model, test_loader)
    
    del model
    gc.collect()


Using model_1.pth

Using model_2.pth

Using model_3.pth

Using model_4.pth

Using model_5.pth


In [32]:
predictions = all_predictions.mean(axis=0)
submission_df.target = predictions
print(submission_df)
#submission_df.to_csv("submission.csv", index=False)

          id    target
0  c0f722661 -0.643145
1  f0953f0a5 -0.414170
2  0df072751 -0.417073
3  04caf4e0c -1.851957
4  0e63f8bea -1.585950
5  12537fe78 -0.971105
6  965e592c0  0.070687


## Upload model

In [33]:
# !mkdir -p ./output/
# !cp -f ./model* ./output/

!mkdir -p /content/drive/MyDrive/CommonLit/output/roberta-large-meanpooling
!content -f /content/model* /content/drive/MyDrive/CommonLit/output/roberta-large-meanpooling

#CHANGEME
!cp -f /content/drive/MyDrive/kaggle/commonlit/pretrained-roberta-base/dataset-metadata.json ./output/dataset-metadata.json
!sed -i -e "s/roberta-base/roberta-large-`TZ=JST-9 date +"%Y%m%d%H%M%S"`/" ./output/dataset-metadata.json
!sed -i -e "s/Roberta-base/Roberta-large-`TZ=JST-9 date +"%m%d%H%M%S"`/" ./output/dataset-metadata.json
!kaggle datasets create -p ./output/

/bin/bash: content: command not found
cp: cannot stat '/content/drive/MyDrive/kaggle/commonlit/pretrained-roberta-base/dataset-metadata.json': No such file or directory
sed: can't read ./output/dataset-metadata.json: No such file or directory
sed: can't read ./output/dataset-metadata.json: No such file or directory
Metadata file not found: dataset-metadata.json


In [30]:
!cat ./output/dataset-metadata.json

cat: ./output/dataset-metadata.json: No such file or directory
